### 專案名稱: 第四屆機器學習百日馬拉松
### 功能描述: 第11天作業
### 版權所有: Dunk  
### 程式撰寫: Dunk  
### 撰寫日期：2020/03/05
### 改版日期:  
### 改版備註:

# 處理 outliers
* 新增欄位註記
* outliers 或 NA 填補
    1. 平均數 (mean)
    2. 中位數 (median, or Q50)
    3. 最大/最小值 (max/min, Q100, Q0)
    4. 分位數 (quantile)

# [作業目標]
- 仿造範例的資料操作, 試著進行指定的離群值處理

# [作業重點]
- 計算 AMT_ANNUITY 的分位點 (q0 - q100) (Hint : np.percentile, In[3])
- 將 AMT_ANNUITY 的 NaN 用中位數取代 (Hint : q50, In[4])
- 將 AMT_ANNUITY 數值轉換到 -1 ~ 1 之間 (Hint : 參考範例, In[5])
- 將 AMT_GOOD_PRICE 的 NaN 用眾數取代 (In[6])

In [48]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# 設定 data_path
dir_data = './data/'

In [49]:
f_app = os.path.join(dir_data, 'application_train.csv')
print('Path of read in data: %s' % (f_app))
app_train = pd.read_csv(f_app)
app_train.head()

Path of read in data: ./data/application_train.csv


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. 列出 AMT_ANNUITY 的 q0 - q100
## 2.1 將 AMT_ANNUITY 中的 NAs 暫時以中位數填補
## 2.2 將 AMT_ANNUITY 的數值標準化至 -1 ~ 1 間
## 3. 將 AMT_GOOD_PRICE 的 NAs 以眾數填補


### ANS:

In [50]:
# 1: 計算 AMT_ANNUITY 的 q0 - q100
import pprint
col_name = 'AMT_ANNUITY'
q_all = [np.percentile(app_train[~app_train[col_name].isnull()][col_name], q = i) for i in np.arange(101)]
df_percent = pd.DataFrame({'q': list(range(101)),
              'value': q_all})
pprint.pprint(df_percent)

       q      value
0      0    1615.50
1      1    6182.91
2      2    6750.00
3      3    7875.00
4      4    8703.00
..   ...        ...
96    96   55624.50
97    97   58482.00
98    98   62964.00
99    99   70006.50
100  100  258025.50

[101 rows x 2 columns]


In [51]:
# 2.1 將 NAs 以 q50 填補
col_name = 'AMT_ANNUITY'
pprint.pprint("Before replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train[col_name].isnull()))

q_50 = df_percent.loc[50,'value']
pprint.pprint("q50 is {0}".format(q_50))
app_train.loc[app_train[col_name].isnull(),col_name] = q_50

pprint.pprint("After replace NAs, numbers of row that AMT_ANNUITY is NAs: %i" % sum(app_train[col_name].isnull()))

'Before replace NAs, numbers of row that AMT_ANNUITY is NAs: 12'
'q50 is 24903.0'
'After replace NAs, numbers of row that AMT_ANNUITY is NAs: 0'


### Hints: Normalize function (to -1 ~ 1)
$ y = 2*(\frac{x - min(x)}{max(x) - min(x)} - 0.5) $

In [52]:
# 2.2 Normalize values to -1 to 1
col_name = 'AMT_ANNUITY'
col_name_add = 'AMT_ANNUITY_NORMALIZED'
print("== Original data range ==")
print(app_train[col_name].describe())

def normalize_value(x):    
    x = ((x-x.min())/(x.max()-x.min())-0.5)*2
    return x

app_train[col_name_add] = normalize_value(app_train[col_name])

print("== Normalized data range ==")
print(app_train[col_name_add].describe())

== Original data range ==
count    307511.000000
mean      27108.487841
std       14493.461065
min        1615.500000
25%       16524.000000
50%       24903.000000
75%       34596.000000
max      258025.500000
Name: AMT_ANNUITY, dtype: float64
== Normalized data range ==
count    307511.000000
mean         -0.801154
std           0.113049
min          -1.000000
25%          -0.883714
50%          -0.818357
75%          -0.742752
max           1.000000
Name: AMT_ANNUITY_NORMALIZED, dtype: float64


In [53]:
# 3 將 AMT_GOOD_PRICE 的 NAs 以眾數填補
from collections import defaultdict

col_name = 'AMT_GOODS_PRICE'
print("Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train[col_name].isnull()))
pprint.pprint(app_train[col_name].unique())
# 列出重複最多的數值
mode_dict = defaultdict(lambda:0)

for value in app_train[~app_train[col_name].isnull()][col_name]:
    mode_dict[value] += 1
    
mode_get = sorted(mode_dict.items(), key=lambda kv: kv[1], reverse=True)
value_most = mode_get[0]

print('重複最多的數值:{0}'.format(value_most))

#驗證眾數
print("== 眾數取代NAs前 ==")
print(any(app_train[col_name]==value_most[0]))
print(len(app_train[app_train[col_name]==value_most[0]]))

app_train.loc[app_train[col_name].isnull(), col_name] = value_most[0]

print("After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: %i" % sum(app_train[col_name].isnull()))

#驗證眾數
print("== 眾數取代NAs後 ==")
print(any(app_train[col_name]==value_most[0]))
print(len(app_train[app_train[col_name]==value_most[0]]))

Before replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 278
array([ 351000. , 1129500. ,  135000. , ...,  453465. ,  143977.5,
        743863.5])
重複最多的數值:(450000.0, 26022)
== 眾數取代NAs前 ==
True
26022
After replace NAs, numbers of row that AMT_GOODS_PRICE is NAs: 0
== 眾數取代NAs後 ==
True
26300
